In [1]:
import pandas as pd
import feather

from keras.models import Sequential
from keras.layers import Dense
from keras.utils.np_utils import to_categorical
from keras.models import load_model


Using TensorFlow backend.


In [2]:
n_groups_per_col = 9
architecture = "2_dense_512"
spread = "spread"
path = "../data/models/ind/binary/" + architecture +"/" + spread + "/"

data_train = pd.read_feather(path + "data_train.feather")

labels_low_train = pd.read_feather(path + "labels_low_train.feather")
labels_high_train = pd.read_feather(path + "labels_high_train.feather")
labels_close_train = pd.read_feather(path + "labels_close_train.feather")

In [3]:
train_data = data_train.to_numpy()
labels_low_train = labels_low_train.to_numpy()
labels_high_train = labels_high_train.to_numpy()
labels_close_train = labels_close_train.to_numpy()

In [4]:
def fit_ind_binary_model(data_train, labels_train):
    
    model = Sequential()
    model.add(Dense(units=512, activation='relu', input_shape=(data_train.shape[1],)))
    model.add(Dense(units=512, activation='relu'))
    model.add(Dense(labels_train.shape[1], activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    history = model.fit(data_train, labels_train, validation_split=0.2, batch_size=512, epochs=3, verbose=1)
    
    return (history, model)

In [8]:
# low
(low_history, low_model) = fit_ind_binary_model(data_train, labels_low_train)
feather.write_dataframe(pd.DataFrame.from_dict(low_history.history), path + "low_history.feather", compression = "uncompressed")
low_model.save(path + "low_model.hdf5")


Train on 3566135 samples, validate on 891534 samples
Epoch 1/3
3566135/3566135 [==============================] - 30s 8us/step - loss: 0.4758 - accuracy: 0.7688 - val_loss: 0.4158 - val_accuracy: 0.7981
Epoch 2/3
3566135/3566135 [==============================] - 29s 8us/step - loss: 0.4236 - accuracy: 0.7948 - val_loss: 0.4307 - val_accuracy: 0.7911
Epoch 3/3
3566135/3566135 [==============================] - 29s 8us/step - loss: 0.4100 - accuracy: 0.8031 - val_loss: 0.3918 - val_accuracy: 0.8146


In [5]:
# high
(high_history, high_model) = fit_ind_binary_model(train_data, labels_high_train)
feather.write_dataframe(pd.DataFrame.from_dict(high_history.history), path + "high_history.feather", compression = "uncompressed")
high_model.save(path + "high_model.hdf5")

Train on 3566135 samples, validate on 891534 samples
Epoch 1/3
3566135/3566135 [==============================] - 29s 8us/step - loss: 0.4824 - accuracy: 0.7671 - val_loss: 0.4246 - val_accuracy: 0.7937
Epoch 2/3
3566135/3566135 [==============================] - 29s 8us/step - loss: 0.4319 - accuracy: 0.7902 - val_loss: 0.4951 - val_accuracy: 0.7488
Epoch 3/3
3566135/3566135 [==============================] - 29s 8us/step - loss: 0.4167 - accuracy: 0.7996 - val_loss: 0.4550 - val_accuracy: 0.7693


In [6]:
# close
(close_history, close_model) = fit_ind_binary_model(train_data, labels_close_train)
feather.write_dataframe(pd.DataFrame.from_dict(close_history.history), path + "history.feather", compression = "uncompressed")
close_model.save(path + "close_model.hdf5")


Train on 3566135 samples, validate on 891534 samples
Epoch 1/3
3566135/3566135 [==============================] - 30s 8us/step - loss: 0.4982 - accuracy: 0.7553 - val_loss: 0.4797 - val_accuracy: 0.7563
Epoch 2/3
3566135/3566135 [==============================] - 30s 8us/step - loss: 0.4684 - accuracy: 0.7655 - val_loss: 0.4582 - val_accuracy: 0.7721
Epoch 3/3
3566135/3566135 [==============================] - 30s 8us/step - loss: 0.4614 - accuracy: 0.7698 - val_loss: 0.4546 - val_accuracy: 0.7731


In [ ]:
### Predict

# low_model = load_model("../data/models/ind/binary/low_model.hdf5")
# high_model = load_model("../data/models/ind/binary/high_model.hdf5")
# close_model = load_model("../data/models/ind/binary/close_model.hdf5")

In [9]:
data_test = pd.read_feather(path + "data_test.feather").to_numpy()

low_pred_prob = low_model.predict_proba(data_test, batch_size = 512)
feather.write_dataframe(pd.DataFrame(low_pred_prob), path + "low_pred_prob.feather", compression='uncompressed')

high_pred_prob = high_model.predict_proba(data_test, batch_size = 512)
feather.write_dataframe(pd.DataFrame(high_pred_prob), path + "high_pred_prob.feather", compression='uncompressed')

close_pred_prob = close_model.predict_proba(data_test, batch_size = 512)
feather.write_dataframe(pd.DataFrame(close_pred_prob), path + "close_pred_prob.feather", compression='uncompressed')
